# 1. Limpieza del texto

Guillermo Luigui Ubaldo Nieto Angarita

## 1.1 Cargar librerías

In [41]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import stanza
import nltk
import fitz  # PyMuPDF

from transformers import AutoTokenizer
from nltk.corpus import stopwords


## 1.2 Descargar los tokenizadores

In [2]:
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline("es")
nlp_spacy = spacy.load("es_core_news_sm")  #cargar el modelo en español
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-08-15 23:58:58 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-15 23:58:58 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-15 23:58:59 INFO: File exists: C:\Users\guill\stanza_resources\es\default.zip
2025-08-15 23:59:02 INFO: Finished downloading models and saved to C:\Users\guill\stanza_resources
2025-08-15 23:59:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-15 23:59:02 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-15 23:59:03 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined     

Descargar y cargar los tokenizadores de NLTK, Stanza, spaCy y BERT.

## 1.2 Descargar Stopwords en Español

In [42]:
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.2 Cargar texto

In [3]:
pdf_path = "texts/certificado.pdf"

## 1.3 Eliminar Encabezado

### 1.3.1 Utilizar pdfplumber

In [4]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                print(text)
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

### 1.3.2 Utilizar PyMuPDF

In [5]:
def extract_text_without_header(pdf_path):
    ignore_texts = {
        "Cámara de Comercio de Barranquilla\nCERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\nDE INSCRIPCION DE DOCUMENTOS.\nFecha de expedición: 24/10/2024 - 13:11:06\n",
        "Recibo No. 12264474, Valor: 7,900\nCODIGO DE VERIFICACIÓN: YK5CB09DFF\n",
        "Página 5 de 5\n"
    }

    remove_texts = {
        "MATRICULA NO RENOVADA",
        "Actualice su registro y evite sanciones",
        """------------------------------------------------------------------------------- 
Verifique  el  contenido  y  confiabilidad  de  este  certificado,  ingresando a
www.camarabaq.org.co/  y digite el código, para que visualice la imagen generada
al  momento  de  su  expedición.  La  verificación  se  puede realizar de manera
ilimitada,  durante  60  días  calendario  contados  a  partir de la fecha de su
expedición.                                                                     
--------------------------------------------------------------------------------""",
        """**********************************************************************
*                                                                    *
*  ATENCION:. ESTE COMERCIANTE NO HA CUMPLIDO CON SU DEBER LEGAL      *
*            DE RENOVAR SU MATRICULA MERCANTIL.                      *
*                                                                    *
**********************************************************************"""
    }


    doc = fitz.open(pdf_path)
    clean_pages = []
    
    for page in doc:
        blocks = page.get_text("blocks")
        page_text = []
        for b in blocks:
            text = b[4]
            # Skip if text matches one of the ignore strings
            if text in ignore_texts:
                continue
            page_text.append(text.strip())
        clean_pages.append("\n".join(page_text))

    clean_text = "\n".join(clean_pages)

    for remove_text in remove_texts:
        clean_text = re.sub(re.escape(remove_text), "", clean_text, flags=re.DOTALL)
        clean_text = clean_text.lstrip("\n")

    
    clean_text = re.sub("C E R T I F I C A", "CERTIFICA", clean_text, flags=re.DOTALL)
    clean_text = re.sub(r"Página\s+\d+\s+de\s+\d+", "", clean_text)
    clean_text = re.sub(r'[ ]{2,}', ' ', clean_text)
    clean_text = re.sub(r'\n[ \t]*\n+', '\n', clean_text)
    #print(clean_text)

    return clean_text

Esta función elimina el encabezado, descarta textos no deseados, reformatea el contenido, suprime la numeración de páginas y corrige espacios y saltos de línea.

## 1.4 Conversión a minúsculas

In [ ]:
def convertToLowerCase(text):
    return text.lower() 

Convertir todo el texto a minúsculas.

## 1.5 Transformación UNICODE

In [ ]:
def unicodeTransformation(text, method):
    return unicodedata.normalize(method, text) 

Normalizar caracteres Unicode.

## 1.6 Tokenización

In [ ]:
def spacy_tokenization(text):
    return [token.text for token in nlp_spacy(text)]

def nltk_tokenization(text):
    return nltk.word_tokenize(text, language='spanish')

def stanza_tokenization(text):
    doc = nlp_stanza(text)
    return [word.text for sent in doc.sentences for word in sent.words]

def bert_tokenization(text):
    max_length = 512
    tokens_bert = tokenizer_bert.tokenize(text)
    chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]
    tokens_flat = [tok for chunk in chunks for tok in chunk]
    return tokens_flat

Aquí se presentan cuatro funciones para tokenizar texto utilizando SpaCy, NLTK, Stanza y BERT.

## 1.7 Eliminación de stop words

In [ ]:
def remove_stopwords_nltk(tokens):
    """
    Remove Spanish stopwords using NLTK.
    
    Parameters:
        tokens (list of str): Tokenized text
    
    Returns:
        list of str: Tokens without stopwords
    """
    return [token for token in tokens if token.lower() not in spanish_stopwords]

def remove_stopwords_spacy(tokens):
    """
    Remove Spanish stopwords using spaCy.
    
    Parameters:
        tokens (list of str): Tokenized text
    
    Returns:
        list of str: Tokens without stopwords
    """
    doc = nlp_spacy(" ".join(tokens))  # rejoin tokens to let spaCy process them
    return [token.text for token in doc if not token.is_stop]

Aquí se incluyen dos funciones para eliminar stopwords, una con NLTK y otra con SpaCy.

## 1.9 Eliminación de elementos no desados

In [52]:
def preprocess_text(text):
    text_lower = convertToLowerCase(text) 

    text_unicode = unicodeTransformation(text_lower,  "NFKD")
    
    tokens = stanza_tokenization(text_unicode)

    tokens_with_stop_words_removed = remove_stopwords_nltk(tokens)
    return tokens_with_stop_words_removed

En este código, primero se toma un texto, se convierte a minúsculas, se aplica la transformación Unicode, se tokeniza con Stanza y, finalmente, se eliminan las stopwords.

## 1.10 Executar el código

In [53]:
raw_text = extract_text_without_header(pdf_path) 

In [55]:
 # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['"', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'negocios', '.', 'renueve', 'matricula', 'mercantil', 'mas', 'tardar', '31', 'marzo', '"', 'fundamento', 'matrícula', 'inscripciones', 'efectuadas', 'registro', 'mercantil', ',', 'cámara', 'comercio', 'certifica', ':', 'certifica', 'nombre', ',', 'identificación', 'domicilio', 'razón', 'social', ':', 'inversiones', '&', 'comercializadora', 'sandoval', 'orozco', 's.a.s.', 'sigla', ':', 'nit', ':', '900.926.396', '-', '0', 'domicilio', 'principal']


## 1.11 Guardar el achievo como un txt

In [56]:
with open("texts/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))

## 1.12 Preguntas

### 1.12.1 ¿Se deben eliminar los StopWords? o Solo algunas

Considero que es mejor eliminarlas ya hay suficiente texto.

### 1.12.2 ¿Qué hacemos con los números?

Apliqué la librería Stanza porque conserva los números de forma intacta.

### 1.12.3 Investigue si existen otras formas de eliminar los encabezados

#### 1.12.4 ¿qué librerías existen?

| Biblioteca                                                                                   | Cómo ayuda a eliminar encabezados                                                                                                                         | Pros                                                                        | Contras                                                                 |
| -------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------- | ----------------------------------------------------------------------- |
| **[pdfplumber](https://github.com/jsvine/pdfplumber)**                                       | Extrae texto con un diseño preciso. Puedes detectar la primera línea de cada página (encabezado) o bloques en la parte superior y excluirlos.             | Fácil para filtrar texto rápidamente. Funciona bien con PDFs estructurados. | No funciona bien con PDFs muy desordenados o escaneados.                |
| **[PyMuPDF (fitz)](https://pymupdf.readthedocs.io/)**                                        | Permite extraer texto por coordenadas de caja delimitadora, por lo que puedes omitir el contenido del margen superior donde suelen estar los encabezados. | Alto control sobre posiciones y elementos.                                  | Requiere conocer las coordenadas o patrones del encabezado.             |
| **[PDFMiner.six](https://github.com/pdfminer/pdfminer.six)**                                 | Da acceso de bajo nivel a la posición del texto; puedes descartar cualquier texto dentro de ciertos rangos de coordenadas *y*.                            | Muy preciso.                                                                | API más compleja que pdfplumber.                                        |
| **[pikepdf](https://github.com/pikepdf/pikepdf)**                                            | Trabaja a nivel de objetos PDF — permite eliminar objetos de encabezado (si están separados).                                                             | Útil para PDFs con una estructura de objetos consistente.                   | No es bueno para detección de texto en línea; requiere conocer objetos. |
| **[Camelot](https://camelot-py.readthedocs.io/)** / **[Tabula](https://tabula.technology/)** | Diseñadas para tablas, pero permiten segmentar páginas e ignorar filas de encabezado en los datos extraídos.                                              | Excelentes para PDFs tabulares.                                             | No sirven para documentos de texto general.                             |
| **[pdfrw](https://github.com/pmaupin/pdfrw)**                                                | Permite manipular objetos de página y potencialmente eliminar capas de encabezado.                                                                        | Puede editar PDFs directamente.                                             | Bajo nivel, no orientada al texto.                                      |


#### 1.12.5 ¿Cómo se pueden aplicar?

##### 1.12.5.1 pdfplumber

In [ ]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                print(text)
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

##### 1.12.5.2 PyMuPDF (fitz)

In [5]:
import pymupdf
doc = pymupdf.open("texts/certificado.pdf")

ignore_texts = ""

for page in doc:
    blocks = page.get_text("blocks")
    page_text = []
    for b in blocks:
        text = b[4]
        # Skip if text matches one of the ignore strings
        if text in ignore_texts:
            continue
        page_text.append(text.strip())
        # print(b)

##### 1.12.5.3 PDFMiner.six

In [6]:
from pdfminer.high_level import extract_text

text = extract_text("texts/certificado.pdf")
print(text)

Cámara de Comercio de Barranquilla
CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O
DE INSCRIPCION DE DOCUMENTOS.
Fecha de expedición: 24/10/2024 - 13:11:06
Recibo No. 12264474, Valor: 7,900
CODIGO DE VERIFICACIÓN: YK5CB09DFF

------------------------------------------------------------------------------- 
Verifique  el  contenido  y  confiabilidad  de  este  certificado,  ingresando a
www.camarabaq.org.co/  y digite el código, para que visualice la imagen generada
al  momento  de  su  expedición.  La  verificación  se  puede realizar de manera
ilimitada,  durante  60  días  calendario  contados  a  partir de la fecha de su
expedición.                                                                     
--------------------------------------------------------------------------------
**********************************************************************
*                                                                    *
*  ATENCION:. ESTE COMERCIANTE NO HA CUMPLIDO CON SU DEBER LEG

##### 1.12.5.4 pikepdf

In [8]:
import pikepdf

with pikepdf.open("texts/certificado.pdf") as pdf:
    num_pages = len(pdf.pages)
    for page in pdf.pages:
        print(page)

<pikepdf.Page({
  "/Contents": pikepdf.Stream(owner=<...>, data=b'q\nq\n0.86603 0.5 -0.5'..., {
    "/Filter": "/FlateDecode",
    "/Length": 1525
  }),
  "/MediaBox": [ 0, 0, 595, 842 ],
  "/Parent": <reference to /Pages>,
  "/Resources": {
    "/Font": {
      "/F1": {
        "/BaseFont": "/Courier",
        "/Encoding": "/WinAnsiEncoding",
        "/Subtype": "/Type1",
        "/Type": "/Font"
      },
      "/F2": {
        "/BaseFont": "/Courier-Bold",
        "/Encoding": "/WinAnsiEncoding",
        "/Subtype": "/Type1",
        "/Type": "/Font"
      },
      "/F3": {
        "/BaseFont": "/Courier-Oblique",
        "/Encoding": "/WinAnsiEncoding",
        "/Subtype": "/Type1",
        "/Type": "/Font"
      }
    },
    "/ProcSet": [ "/PDF", "/Text", "/ImageB", "/ImageC", "/ImageI" ],
    "/XObject": {
      "/Xf1": pikepdf.Stream(owner=<...>, data=<...>, {
        "/BBox": [ 0, 0, 30, <...> ],
        "/Filter": <...>,
        "/FormType": <...>,
        "/Length": <...>,
   

##### 1.12.5.5 PdfReader

In [9]:
from PyPDF2 import PdfReader

pdf_path = "texts/certificado.pdf"

reader = PdfReader(pdf_path)
for i, page in enumerate(reader.pages, start=1):
    text = page.extract_text()
    print(f"--- Page {i} ---")
    print(text)


--- Page 1 ---
MATRICULA NO RENOVADA
Actualice su registro y evite sanciones
------------------------------------------------------------------------------- 
Verifique  el  contenido  y  confiabilidad  de  este  certificado,  ingresando a
www.camarabaq.org.co/  y digite el código, para que visualice la imagen generada
al  momento  de  su  expedición.  La  verificación  se  puede realizar de manera
ilimitada,  durante  60  días  calendario  contados  a  partir de la fecha de su
expedición.                                                                     
--------------------------------------------------------------------------------
**********************************************************************
*                                                                    *
*  ATENCION:. ESTE COMERCIANTE NO HA CUMPLIDO CON SU DEBER LEGAL      *
*            DE RENOVAR SU MATRICULA MERCANTIL.                      *
*                                                                    *


##### 1.12.5.6 Camelot

In [ ]:
import camelot

#### 1.12.6 ¿En qué eventos los podemos implementar?